#Scheduling electrical vehicles that travel a segment on a single road

Accompanying code to the thesis of Jurriaan Berger, under supervision of Matus Mihalak.

In [ ]:
!pip install ortools
from ortools.graph import pywrapgraph

In [1]:
import numpy as np
import time
import pickle

In [134]:
#Relative load per road segment, based on 24hrs:
rel_load = [0.16, 0.12, 0.15, 0.13, 0.11, 0.06, 0.07, 0.06, 0.09, 0.07]

#Total number of charging stations:
total_stations = 170

#Number of charging stations per road segment:
segment_stations = [int(r*total_stations) for r in rel_load]

unique_rel_dates = 96
max_completion_time = 20
max_schedule_length = unique_rel_dates+max_completion_time
max_trip_length = len(rel_load)

##Loading Data and generate trips

In [3]:
# Read in the file that contains the traffic flow for every 25 km, every 15 minutes
with open("traffic_flow.csv", encoding='utf-8-sig') as file_name:
    array = np.loadtxt(file_name, delimiter=";")

In [ ]:
# In case you only want to run the algorithms for a smaller timeframe, run this code cell
new_array = np.array(array)
array = new_array[0:10,20:30]
array = array.astype(int)

In [4]:
# Take only 10% of the traffic flow
capacity = array/10
capacity = capacity.astype(int)

In [ ]:
# Generate the trips
# In the following way: start a trip at t=0, s=0, take one from the capacity at this time-space
# Repeat for this trip at t+1,s+1, untill the end of the road is reached or there is a time-space point that has no capacity anymore
# Continue until there is no capacity at t=0, s=0, then go to t=0, s=1
# Until all time-space points are visited and the left-over capacity is 0 everywhere

trips = {}
trip_counter = 0

for i in range(len(capacity)):
  for j in range(len(capacity[0])):
    while capacity[i][j] > 0:

      begin = [i,j]
      end = [i,j]
      capacity[i][j]-=1
      length = 1
      bi = i+1
      bj = j+1

      while bi < len(capacity) and bj < len(capacity[0]) and capacity[bi][bj]>0:
        capacity[bi][bj]-=1
        end = [bi,bj]
        length+=1
        bi+=1
        bj+=1
        
      trips[trip_counter] = {}
      trips[trip_counter]['rel_date'] = j
      trips[trip_counter]['start_seg'] = i
      trips[trip_counter]['final_seg'] = bi-1
      trips[trip_counter]['length']=length

      trip_counter+=1

print('Created',trip_counter,'trips')


In [6]:
#Store the trips as separate file, for re-use

a_file = open("data.pkl", "wb")
pickle.dump(trips, a_file)
a_file.close()

In [ ]:
# Load trips from file if required

a_file = open("data.pkl", "rb")
trips = pickle.load(a_file)
print(len(trips))

## Setup the graph for the minimum cost flow problem

In [135]:
n = len(trips) #number of jobs

m = segment_stations      #number of machines per segment, this

c = max_completion_time   #max completion time
r = unique_rel_dates      #number of distinct release dates
l = max_trip_length       #max trip length 
rc = r + c + l - 2        #the number of timeslots for every machine

#Graph structure:
#source > jobs > timeslots (only edges to reachable timeslots, regarding time and space) > sink

In [ ]:
#START NODES

#an edge from the source to every job 
start_source = np.array(np.zeros(n).astype(int)).tolist()

#an edge from every job to all possible timeslots (for every segment, number of machines*)
start_jobs = []

cnt = 0

for t in trips:
  cnt+=1
  if cnt%500 is 0:
    print(cnt)

  t_start = []
  for i in range(trips[t]['start_seg'],trips[t]['final_seg']+1):
    t_start = np.append(t_start,np.ones(m[i]*c)*(t+1))
  
  lt_start = t_start.astype(int).tolist()
  start_jobs.extend(lt_start)
    
#an edge from all possible timeslots to the sink
start_ts = list(range(n+1,n+1+sum(m)*rc))

In [ ]:
#END NODES

#an edge from the source to every job
end_source = list(range(1,n+1))

#an edge from every job to all allowed timeslots
end_jobs = []
cnt=0

for t in trips:
  cnt+=1
  if cnt%500 is 0: 
    print(cnt)

  ts_before_segment = 0
  for k in range(0,trips[t]['start_seg']): ts_before_segment+=(m[k]*rc)-1 
  
  for i in range(trips[t]['start_seg'],trips[t]['final_seg']+1):
    arr_times = []
    
    for j in range(m[i]):                               #for all machines (hence the +j*rc)
      arr_times.extend(list(range(i+j*rc,(i+j*rc)+c)))  #for a machines in the segment, only at the allowed arrival times: 
                                                        #(0,1,...,max_completion time) for the first charging station
                                                        #(1,2,...,max_completion time+travel_distance) for the second charging station, etc.
    
    arr_times = [a + int(trips[t]['rel_date'])    #add the release date
                 + ts_before_segment              #increase by the number of timeslots on all previous machines
                 + n+1                            #increase by the number of jobs (to account for the numbers taken by the other nodes)
                 for a in arr_times] 
    
    ts_before_segment += m[i]*rc
    end_jobs.extend(arr_times)

if(len(end_jobs) - len(start_jobs) is not 0): print('Different number of start/end points for jobs, problem!',len(end_jobs),len(start_jobs))

#an edge from all possible timeslots to the sink
end_ts = np.ones(len(start_ts))*(max(start_ts)+1)
end_ts = end_ts.astype(int).tolist()

In [138]:
#CAPACITIES (one for all edges)

cap_source = np.array(np.ones(n).astype(int)).tolist()
cap_jobs = np.array(np.ones(len(start_jobs)).astype(int)).tolist()
cap_ts = np.array(np.ones(len(start_ts)).astype(int)).tolist()

In [139]:
#COSTS

#Cost = 1 for all edges from source
cost_source = np.array(np.zeros(n).astype(int)).tolist()

#Cost = waiting time for jobs = 1,2,3,4,...,10 to every machine
cost_jobs = []

for t in trips:
  for i in range(trips[t]['start_seg'],trips[t]['final_seg']+1):
    for j in range(m[i]):
      cost_jobs.extend(list(range(1,c+1)))

#Cost = 1 for all edges from timeslots to sink
cost_ts = np.array(np.zeros(len(start_ts)).astype(int)).tolist()

In [140]:
#SUPPLIES

supplies = [n]

no_supplies = np.array(np.zeros(end_ts[0]-1).astype(int)).tolist()
supplies.extend(no_supplies)
supplies.append(-n)

In [ ]:
# Decoder for vertices (not required):

node_to_timeslot = {}

segment = 0
slots_prev_segment = 0
slots_this_segment = m[segment]*rc+slots_prev_segment

for t in sorted(set(end_jobs)):
  timeslot = t-(n+1) # subtract the number of nodes that were used by the source and job nodes  
  
  if(slots_prev_segment <= timeslot < slots_this_segment): this_segment=segment #print('timeslot',timeslot,'in segment',segment)
  else:
    slots_prev_segment = slots_this_segment
    segment +=1
    slots_this_segment = m[segment]*rc+slots_prev_segment

    this_segment=segment
    
  this_machine = int((timeslot-slots_prev_segment)/rc)  
  this_time = timeslot%rc #for all machines there is a constant number of timeslots per machine

  print('t:',t,'timeslot:',timeslot)
  print('segment', this_segment, 'machine', this_machine,'time', this_time)
  timeslot_loc = {'segment': this_segment, 'machine': this_machine,'time': this_time}
  node_to_timeslot[t]=timeslot_loc

## LP solver minimum cost flow problem

Based on: https://developers.google.com/optimization/flow/assignment_min_cost_flow

In [141]:
# Instantiate a SimpleMinCostFlow solver.
min_cost_flow = pywrapgraph.SimpleMinCostFlow()

In [142]:
# Define the directed graph for the flow. (Get this from the data automatically, see below)
start_nodes = start_source + start_jobs + start_ts
end_nodes = end_source + end_jobs + end_ts
capacities = cap_source + cap_jobs + cap_ts
costs = (cost_source + cost_jobs + cost_ts)

source = 0
sink = end_ts[0]
tasks = n
supplies = supplies #defined in section above

In [ ]:
# Add each arc.
cnt=0

for i in range(len(start_nodes)):
    cnt+=1
    if cnt%100000 is 0: print(cnt)
    min_cost_flow.AddArcWithCapacityAndUnitCost(start_nodes[i],
                                                end_nodes[i], capacities[i],
                                                costs[i])

# Add node supplies.
for i in range(len(supplies)):
    min_cost_flow.SetNodeSupply(i, supplies[i])

In [ ]:
# Find the minimum cost flow
start = time.time()
status = min_cost_flow.Solve()
end = time.time()

print('Time:',end-start)

In [ ]:
# Get total and maximum waiting time

max_wait = 0
cnt = 0

if status == min_cost_flow.OPTIMAL:
    print('Total cost = ', min_cost_flow.OptimalCost())
    for arc in range(min_cost_flow.NumArcs()):
        # Can ignore arcs leading out of source or into sink.
        if min_cost_flow.Tail(arc) != source and min_cost_flow.Head(
                arc) != sink:

            # Arcs in the solution have a flow value of 1. Their start and end nodes
            # give an assignment of job to a timeslot.
            if min_cost_flow.Flow(arc) > 0: 
              cnt+=1
              if max_wait < min_cost_flow.UnitCost(arc):
                max_wait = min_cost_flow.UnitCost(arc)
    print('maximum waiting time:',max_wait)
    print('num:',cnt)
else:
    print('There was an issue with the min cost flow input.')
    print(f'Status: {status}')

## LFJ and Greedy Scheduling algorithms

In [38]:
# Function to find the machine where the job gets the shortest waiting time

def schedule_job(current_schedule, job_j):
  shortest_waiting_time = 1000000
  shortest_waiting_machine = []
  shortest_waiting_interval = -1
  shortest_segment = -1
  shortest_machine = -1

  segment_ind = 0

  for segment in range(job_j['start_seg'],job_j['final_seg']+1):
    #print('segment ',segment)
    machine_ind = 0
    
    for machine in current_schedule[segment]:
      #print('machine ',machine)
      found = False
      waiting_time = 1
      i = job_j['rel_date']+segment_ind 

      while not found and i < len(machine) and waiting_time<shortest_waiting_time:
        #print('i ',i, machine[i])
        
        if machine[i] == 0:
          #print('Timeslot found')
          found = True
          shortest_waiting_time = waiting_time
          shortest_waiting_machine = machine
          shortest_waiting_interval = i
          shortest_segment = segment
          shortest_machine = machine_ind

        i+=1; waiting_time+=1
        #print('WT:',waiting_time)
      machine_ind+=1
    segment_ind+=1


  #print('Scheduled ',job_j,'on segment, machine',[shortest_segment,shortest_machine])
  shortest_waiting_machine[shortest_waiting_interval] = shortest_waiting_time
  job_j['pos_schedule'] = [shortest_segment,shortest_machine,shortest_waiting_interval] #segment ; machine ; timeslot
  job_j['completion_time'] = shortest_waiting_time
        
  return current_schedule

In [117]:
# Initialize the schedules

schedules = {}

for seg in range(len(segment_stations)):
  schedules[seg]= np.array(np.zeros((segment_stations[seg],max_schedule_length))).astype(int)

In [118]:
# Set the number of machines that are available for a job

for t in trips:
  machines_t=0
  for s in range(trips[t]['start_seg'],trips[t]['final_seg']+1):
    machines_t += segment_stations[s]
  trips[t]['machine_count'] = machines_t

In [ ]:
# Get all the unique sizes of the eligibility sets (number of machines that can process a job)

unique_machine_count = []

for t in trips:
  if trips[t]['machine_count'] not in unique_machine_count: unique_machine_count.append(trips[t]['machine_count'])

print(sorted(unique_machine_count))

In [ ]:
# Greedy scheduling

greedy_total_completion_time=0
greedy_max_completion_time=0

for t in range(len(trips)):
  schedule_job(schedules,trips[t])
  greedy_total_completion_time+=trips[t]['completion_time']
  if trips[t]['completion_time'] > greedy_max_completion_time: greedy_max_completion_time=trips[t]['completion_time']

print('Total waiting time greedy',greedy_total_completion_time)
print('Max waiting time',greedy_max_completion_time)

In [ ]:
# LFJ scheduling 

lfj_total_completion_time = 0
lfj_max_completion_time = 0

start=time.time()

for f in sorted(unique_machine_count):
  #print(f)
  for p in trips:
    if trips[p]['machine_count']==f:
      schedules = schedule_job(schedules,trips[p])
      lfj_total_completion_time+=trips[p]['completion_time']
      if trips[p]['completion_time'] > lfj_max_completion_time: lfj_max_completion_time=trips[p]['completion_time']

end=time.time()

print('TIME:',end-start)
print('Total waiting time LFJ',lfj_total_completion_time)
print('Max waiting time LFJ',lfj_max_completion_time)

## Generate random trips

In [ ]:
random_trips = {}
number_trips = 5000

max_trip_length = 9 #10-1
distinct_rel_dates = 50

np.random.seed(16)

In [ ]:
# First generate the lengths of the trips (normal distribution, between 1 and 20)

mean_length = 0.5*max_trip_length
std_length = 1.5

lengths = np.random.normal(mean_length,std_length,number_trips).astype(int)

# Make sure the generated trips fall within the bounds of length 1 to max-length
for l in range(len(lengths)):
  if lengths[l] < 1: 
    lengths[l] = 1
  if lengths[l] >max_trip_length: 
    lengths[l] =max_trip_length

In [ ]:
# Second generate the start point of the trip (uniform)

for i in range(number_trips):
  random_trips[i] = {}
  random_trips[i]['length'] = lengths[i]
  start = np.random.default_rng().integers(max_trip_length+1-lengths[i])
  random_trips[i]['start_seg'] = start
  random_trips[i]['final_seg'] = start+lengths[i]
  if start+lengths[i]-1 > max_trip_length: print(start, lengths[i])


In [ ]:
# Third generate the release date of the trip

for r in random_trips:
  random_trips[r]['rel_date'] = np.random.default_rng().integers(distinct_rel_dates)

In [ ]:
trips=random_trips